## Load packages

In [1]:
import numpy as np
import pandas as pd
import math
from keras import layers
from keras.layers import *
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import multilabel_confusion_matrix, classification_report

%matplotlib inline

Using TensorFlow backend.
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/magu/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
# set number of cores to 8
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=8, 
                                                   inter_op_parallelism_threads=8)))

## Load dataset

In [3]:
# load processed genetic data from 1000 Genomes
chr1 = np.load('/home/magu/deepmix/data/ALL_DNA_dataset/chr1_1kg_X.npz')

# create dict mapping from sampleid to superpopulation
sample_info = pd.read_csv('/home/jsokol/Data/igsr_samples.tsv', delimiter="\t")
sample_to_superpopulation = sample_info.set_index('Sample name')['Superpopulation code'].dropna().to_dict()
# remove missing samples
temp = set()
for x in np.nditer(chr1['S']):
    sample_id = str((x.item(0))).replace("'", "").replace("b", "").split("_")[0]
    if sample_id not in sample_to_superpopulation:
        print(sample_id + ' is not in dict. Make sure to remove this sample from the dataset.')
    elif sample_to_superpopulation[sample_id] == float('nan'):
        print(sample_id + ' maps to nan in dict')
        
# define these for use, print input shape
chr1_V = chr1['V']
X = chr1['G']
X.shape

(5008, 57876, 4)

In [4]:
# define number of output classes
classes = {val:key for key,val in enumerate(set(sample_to_superpopulation.values()))}
classes

{'EUR': 0, 'EAS': 1, 'AMR': 2, 'SAS': 3, 'AFR': 4}

In [5]:
# one hot encode labels
Y = np.zeros((X.shape[0], len(classes)))
keep=set(range(X.shape[0]))
for i in range(X.shape[0]):
    sample_id = str(chr1['S'][i].item(0)).replace("'", "").replace("b", "").split("_")[0]
    Y[i, classes[sample_to_superpopulation[sample_id]]] = 1
Y[np.array([1,353,2043]), :] # show some random samples

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [6]:
[X.shape, Y.shape]

[(5008, 57876, 4), (5008, 5)]

In [7]:
# randomly shuffle the order of the data
randomize = np.random.choice(X.shape[0], X.shape[0], replace=False)
X = X[randomize,:]
Y = Y[randomize,:]

In [8]:
# split data into training and test sets
ntrain=3000
X_train = X[:ntrain,:,:]
X_test = X[ntrain:,:,:]
Y_train = Y[:ntrain,:]
Y_test = Y[ntrain:,:]
print(X_train.shape, Y_train.shape)

(3000, 57876, 4) (3000, 5)


## Build model

In [9]:
def cnn_cat(shape):
    """
    input_shape: The height, width and channels as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]
    """
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=shape)

    # First convolutional block
    s=64
    X = Conv1D(filters=64, kernel_size=s ,strides=s//4, padding = 'valid', name = 'conv0')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn0')(X)
    X = Activation('relu')(X)

    # Second convolutional block
    X = Conv1D(64, 64, strides = 4, padding = 'same', name = 'conv1')(X)
    X = BatchNormalization(axis = -1, name = 'bn1')(X)
    X = Activation('relu')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(Y.shape[1], activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='cnn_cat')
    
    return model

## Create and compile the model 

In [10]:
# create model
model = cnn_cat(X_train.shape[1:])
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 57876, 4)          0         
_________________________________________________________________
conv0 (Conv1D)               (None, 3614, 64)          16448     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 3614, 64)          256       
_________________________________________________________________
activation_1 (Activation)    (None, 3614, 64)          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 904, 64)           262208    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 904, 64)           256       
_________________________________________________________________
activation_2 (Activation)    (None, 904, 64)           0         
__________

## Train model

In [ ]:
history=model.fit(X_train, Y_train, epochs = 4, batch_size = 128,
                  validation_data=(X_test, Y_test))

Train on 3000 samples, validate on 2008 samples
Epoch 1/4
3000/3000 [==============================] - 117s 39ms/step - loss: 9.3540 - acc: 0.3737 - val_loss: 9.7775 - val_acc: 0.3929
Epoch 2/4
3000/3000 [==============================] - 113s 38ms/step - loss: 9.6924 - acc: 0.3987 - val_loss: 9.7768 - val_acc: 0.3934
Epoch 3/4
3000/3000 [==============================] - 117s 39ms/step - loss: 9.6924 - acc: 0.3987 - val_loss: 9.7768 - val_acc: 0.3934
Epoch 4/4
 896/3000 [=======>......................] - ETA: 1:01 - loss: 9.5341 - acc: 0.4085

## Evaluate model

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.title('Loss')

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.title('Accuracy')

In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(X_test, Y_test, batch_size=1)
print('test loss, test acc:', results)

In [ ]:
test_Y_hat=model.predict(X_test)
result=confusion_matrix(np.argmax(test_Y_hat, axis=-1).flatten(), 
                        np.argmax(Y_test, axis=-1).flatten())
pd.DataFrame(result, index=[k for k,v in sorted(classes.items(), key=lambda x:x[1])], 
                     columns=[k for k,v in sorted(classes.items(), key=lambda x:x[1])])